In [37]:
from flask import Flask, render_template, request, redirect
import requests
import vonage

app = Flask(__name__)

# Replace these with your Vonage API credentials
VONAGE_API_KEY = '9e4e94bd'
VONAGE_API_SECRET = 'lRpsuzim00QVvhoy'
VONAGE_PHONE_NUMBER = '919176000532'

@app.route('/')
def home():
    # Fetch WhatsApp messages
    messages = fetch_whatsapp_messages()
    return render_template('index.html', whatsapp_messages=messages)

def fetch_whatsapp_messages():
    # Use Vonage API to fetch WhatsApp messages
    url = f'https://messages.api.vonage.com/v1/messages'
    headers = {
        'Authorization': f'Basic {VONAGE_API_KEY}:{VONAGE_API_SECRET}'
    }
    params = {
        'to': VONAGE_PHONE_NUMBER,
        'type': 'whatsapp',
    }
    response = requests.get(url, headers=headers, params=params)
    data = response.json()

    # Process and extract relevant information from the messages
    whatsapp_messages = []
    for message in data['_embedded']['messages']:
        whatsapp_messages.append({
            'from': message['from']['type'],
            'content': message['content']['text'],
            'timestamp': message['timestamp'],
            'message_id': message['message-id']
        })

    return whatsapp_messages

@app.route('/send_whatsapp_message', methods=['POST'])
def send_whatsapp_message():
    # Retrieve the text message and recipient phone number entered by the user
    recipient_phone = request.form.get('recipient_phone')
    message = request.form.get('message')

    # Use Vonage API to send WhatsApp message
    url = f'https://messages.api.vonage.com/v1/messages'
    headers = {
        'Authorization': f'Basic {VONAGE_API_KEY}:{VONAGE_API_SECRET}'
    }
    data = {
        'from': {'type': 'whatsapp', 'number': VONAGE_PHONE_NUMBER},
        'to': {'type': 'whatsapp', 'number': recipient_phone},
        'message': {'content': {'type': 'text', 'text': message}}
    }
    response = requests.post(url, headers=headers, json=data)

    # Redirect back to the dashboard
    return redirect('/')

if __name__ == '__main__':
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
[2023-07-16 21:23:21,924] ERROR in app: Exception on / [GET]
Traceback (most recent call last):
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\urllib3\connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\urllib3\util\connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "C:\Users\Sulochana\anaconda3\lib\socket.py", line 954, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno 11001] getaddrinfo failed

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\u

In [5]:
%tb

SystemExit: 1

In [13]:
from flask import Flask, request, jsonify
from vonage import Client, Sms
import os

app = Flask(__name__)

# Set up Vonage API client
api_key = '9e4e94bd'
api_secret = 'lRpsuzim00QVvhoy'
VONAGE_PHONE_NUMBER = '+919176000532'

client = Client(key=api_key, secret=api_secret)
whatsapp_messages = []
# Define the webhook route to handle incoming messages
@app.route('/webhook', methods=['POST'])
def webhook():
    data = request.get_json()

    # Check if it's a WhatsApp message
    if data['type'] == 'whatsapp':
        # Process the incoming WhatsApp message
        sender = data['from']['number']
        message = data['message']['content']['text']

        # Store the incoming message in the list
        whatsapp_messages.append({'sender': sender, 'message': message})

        # Send a response to the incoming message (optional)
        send_whatsapp_message(sender, 'Thank you for your message!')

    return jsonify({'success': True})
# Function to send a WhatsApp message
def send_whatsapp_message(recipient, message):
    whatsapp = Sms(client)
    response = whatsapp.send_message({
        'from': 'YOUR_WHATSAPP_NUMBER',
        'to': recipient,
        'text': message
    })

    return response

# Route to display WhatsApp messages in a webpage
@app.route('/')
def display_messages():
    return render_template('index.html', whatsapp_messages=whatsapp_messages)

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 18:17:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 18:17:24] "GET /retrieve-whatsapp-messages HTTP/1.1" 404 -
127.0.0.1 - - [16/Jul/2023 18:17:41] "POST / HTTP/1.1" 405 -


In [17]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

# List to store received WhatsApp messages
whatsapp_messages = []

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/webhook', methods=['POST'])
def webhook():
    global whatsapp_messages

    # Receive data from the webhook
    data = request.json

    # Extract the message and sender information
    message = data['message']['content']['text']['body']
    sender = data['from']['number']

    # Append the message to the whatsapp_messages list
    whatsapp_messages.append({'sender': sender, 'message': message})

    return jsonify({'status': 'success'})

@app.route('/get_messages', methods=['GET'])
def get_messages():
    global whatsapp_messages
    return jsonify(whatsapp_messages)

if __name__ == "__main__":
    app.run(port=5000)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 18:23:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 18:23:38] "GET /get_messages HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 18:23:51] "POST / HTTP/1.1" 405 -


In [39]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

# List to store received WhatsApp messages
whatsapp_messages = []

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/webhook', methods=['POST'])
def webhook():
    global whatsapp_messages

    # Receive data from the webhook
    data = request.json
    print(data)
    # Extract the message and sender information
    message = data['message']['content']['text']['body']
    sender = data['from']['number']

    # Append the message to the whatsapp_messages list
    whatsapp_messages.append({'sender': sender, 'message': message})

    return jsonify({'status': 'success'})

@app.route('/get_messages', methods=['GET'])
def get_messages():
    global whatsapp_messages
    return jsonify(whatsapp_messages)

if __name__ == "__main__":
    app.run(port=80)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 22:34:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:34:24] "GET /get_messages HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:35:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:35:13] "GET /get_messages HTTP/1.1" 200 -
[2023-07-16 22:35:34,255] ERROR in app: Exception on /webhook [POST]
Traceback (most recent call last):
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Sulochana\anaconda3\li

{'message': {'content': {'text': {'body': 'trying'}}}, 'to': {'type': 'whatsapp', 'number': '919176000532'}}


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 18:40:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 18:40:02] "GET /get_messages HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 18:40:10] "POST / HTTP/1.1" 405 -


In [24]:
#!/usr/bin/env python3
from pprint import pprint
from flask import Flask, request

app = Flask(__name__)


@app.route("/webhooks/inbound-message", methods=["POST"])
def inbound_message():
    data = request.get_json()
    pprint(data)
    return "200"


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 19:02:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 19:02:38] "GET /get_messages HTTP/1.1" 404 -


In [36]:
from flask import Flask, request, jsonify

app = Flask(__name__)

@app.route('/webhook', methods=['POST'])
@app.route("/webhooks/inbound", methods=["POST"])
def inbound():
    # Need token after 'Bearer'
    parts = request.headers["authorization"].split()
    token = parts[1].strip()

    # Extract api_key from token payload
    k = jwt.decode(token, verify=False)["0ruxaxeDfFwRAEzzGiEcPCkvRIwhOkgunLPlgXIwxrH8UNe65Z"]
    # Use k to look up corresponding sig secret

    #### 1. Verify request
    try:
        decoded = jwt.decode(token, sig_secret, algorithms="HS256")
    except Exception as e:
        print(e)
        r = '{"msg": "' + str(e) + '"}'
        return (r, 401)

    #### 2. Verify payload (only needed if using HTTP rather than HTTPS)

    # Obtain transmitted payload hash
    payload_hash = decoded["payload_hash"]

    # generate hash of request payload
    payload = request.data  # Obtains request data as binary string
    h = hashlib.sha256(payload)  # requires binary string
    hd = h.hexdigest()  # Use hexdigest() and NOT digest()

    # Check the payload hash matches the one we created ourselves from request data
    if hd != payload_hash:
        return ('{"msg": "Invalid payload"}', 401)
    else:
        print("Verified payload")
    return "OK"


@app.route("/webhooks/status", methods=["POST"])
def status():
    data = request.get_json()
    return jsonify(data)
    
@app.route('/')
def index():
    return render_template('index.html')

if __name__ == "__main__":
    app.run(port=80)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 21:21:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 21:21:48] "GET /get_messages HTTP/1.1" 404 -
[2023-07-16 21:21:57,353] ERROR in app: Exception on /webhook [POST]
Traceback (most recent call last):
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\Sulochana\anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Sulochana\anaco

In [30]:
@app.route("/webhooks/inbound", methods=["POST"])
def inbound():
    # Need token after 'Bearer'
    parts = request.headers["authorization"].split()
    token = parts[1].strip()

    # Extract api_key from token payload
    k = jwt.decode(token, verify=False)["api_key"]
    # Use k to look up corresponding sig secret

    #### 1. Verify request
    try:
        decoded = jwt.decode(token, sig_secret, algorithms="HS256")
    except Exception as e:
        print(e)
        r = '{"msg": "' + str(e) + '"}'
        return (r, 401)

    #### 2. Verify payload (only needed if using HTTP rather than HTTPS)

    # Obtain transmitted payload hash
    payload_hash = decoded["payload_hash"]

    # generate hash of request payload
    payload = request.data  # Obtains request data as binary string
    h = hashlib.sha256(payload)  # requires binary string
    hd = h.hexdigest()  # Use hexdigest() and NOT digest()

    # Check the payload hash matches the one we created ourselves from request data
    if hd != payload_hash:
        return ('{"msg": "Invalid payload"}', 401)
    else:
        print("Verified payload")
    return "OK"


@app.route("/webhooks/status", methods=["POST"])
def status():
    data = request.get_json()
    return jsonify(data)

In [40]:
from flask import Flask, render_template, request, jsonify

app = Flask(__name__)

# List to store received WhatsApp messages
whatsapp_messages = []

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/webhook', methods=['POST'])
def webhook():
    global whatsapp_messages

    # Receive data from the webhook
    data = request.json
    print(data)

    # Extract the message and sender information safely using data.get()
    message = data['message']['content']['text']['body']
    sender = data.get('from', {}).get('number', '')

    # Append the message to the whatsapp_messages list
    whatsapp_messages.append({'sender': sender, 'message': message})

    return jsonify({'status': 'success'})

@app.route('/get_messages', methods=['GET'])
def get_messages():
    global whatsapp_messages
    return jsonify(whatsapp_messages)

if __name__ == "__main__":
    app.run(port=80)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 22:39:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:39:10] "GET /get_messages HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:39:20] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:39:20] "GET /get_messages HTTP/1.1" 200 -


{'message': {'content': {'text': {'body': 'trying'}}}, 'to': {'type': 'whatsapp', 'number': '919176000532'}}


127.0.0.1 - - [16/Jul/2023 22:40:26] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:40:26] "GET /get_messages HTTP/1.1" 200 -


In [41]:
from flask import Flask, render_template, request, jsonify, redirect

app = Flask(__name__)

# List to store received WhatsApp messages
whatsapp_messages = []

@app.route('/')
def index():
    return render_template('index.html', messages=whatsapp_messages)

@app.route('/webhook', methods=['POST'])
def webhook():
    global whatsapp_messages

    # Receive data from the webhook
    data = request.json
    print(data)

    # Extract the message and sender information safely using data.get()
    message = data['message']['content']['text']['body']
    sender = data.get('from', {}).get('number', '')

    # Append the message to the whatsapp_messages list
    whatsapp_messages.append({'sender': sender, 'message': message})

    return jsonify({'status': 'success'})

@app.route('/get_messages', methods=['GET'])
def get_messages():
    global whatsapp_messages
    return jsonify(whatsapp_messages)

@app.route('/reply/<int:message_index>', methods=['GET', 'POST'])
def reply(message_index):
    if message_index > 0 and message_index <= len(whatsapp_messages):
        # Retrieve the message based on the index
        message = whatsapp_messages[message_index - 1]

        if request.method == 'POST':
            # Handle the reply form submission
            reply_message = request.form.get('reply_message', '')
            # Assuming you have an external function to send the reply.
            # You can replace the print statement with the actual code
            # to send the reply back to the user.
            print(f"Sending reply to {message['sender']}: {reply_message}")
            return redirect('/')

        return render_template('reply.html', message=message)
    else:
        return "Invalid message index"

if __name__ == "__main__":
    app.run(port=80)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 22:45:43] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:45:43] "GET /get_messages HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:45:55] "POST /webhook HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:45:56] "GET /get_messages HTTP/1.1" 200 -


{'message': {'content': {'text': {'body': 'trying'}}}, 'to': {'type': 'whatsapp', 'from': '14157386102', 'number': '919176000532'}}


127.0.0.1 - - [16/Jul/2023 22:46:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:46:01] "GET /get_messages HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:46:05] "GET /reply/1 HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:46:16] "POST /reply/1 HTTP/1.1" 302 -
127.0.0.1 - - [16/Jul/2023 22:46:16] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:46:16] "GET /get_messages HTTP/1.1" 200 -


Sending reply to : how to see is this going or not


127.0.0.1 - - [16/Jul/2023 22:47:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:47:01] "GET /get_messages HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:47:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:47:07] "GET /get_messages HTTP/1.1" 200 -


In [43]:
from flask import Flask, render_template, request, jsonify, redirect
import requests

app = Flask(__name__)

# List to store received WhatsApp messages
whatsapp_messages = []

# Replace these with your Vonage API credentials
API_KEY = "ef666e52"
API_SECRET = "bIU3rJiCg3fO8pqd"
VONAGE_NUMBER = '16473793941'

@app.route('/')
def index():
    return render_template('index.html', messages=whatsapp_messages)

@app.route('/webhook', methods=['POST'])
def webhook():
    global whatsapp_messages

    # Receive data from the webhook
    data = request.json
    print(data)

    # Extract the message and sender information safely using data.get()
    message = data['message']['content']['text']['body']
    sender = data.get('from', {}).get('number', '')

    # Append the message to the whatsapp_messages list
    whatsapp_messages.append({'sender': sender, 'message': message})

    return jsonify({'status': 'success'})

@app.route('/get_messages', methods=['GET'])
def get_messages():
    global whatsapp_messages
    return jsonify(whatsapp_messages)

@app.route('/reply/<int:message_index>', methods=['GET', 'POST'])
def reply(message_index):
    if message_index > 0 and message_index <= len(whatsapp_messages):
        # Retrieve the message based on the index
        message = whatsapp_messages[message_index - 1]

        if request.method == 'POST':
            # Handle the reply form submission
            reply_message = request.form.get('reply_message', '')
            
            # Send the reply using Vonage API
            url = 'https://api.nexmo.com/v0.1/messages'
            headers = {
                'Authorization': f'Basic {API_KEY}:{API_SECRET}',
                'Content-Type': 'application/json',
                'Accept': 'application/json',
            }
            payload = {
                'from': {'type': 'whatsapp', 'number': VONAGE_NUMBER},
                'to': {'type': 'whatsapp', 'number': message['sender']},
                'message': {'content': {'type': 'text', 'text': reply_message}},
            }
            response = requests.post(url, headers=headers, json=payload)

            print(response.status_code)
            print(response.json())
            
            return redirect('/')

        return render_template('reply.html', message=message)
    else:
        return "Invalid message index"

if __name__ == "__main__":
    app.run(port=80)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:80/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jul/2023 22:54:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:54:50] "GET /get_messages HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:55:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2023 22:55:14] "GET /get_messages HTTP/1.1" 200 -
